<a href="https://colab.research.google.com/github/objectin/Goorm-AI-Team-Project/blob/main/mrc_bigbird.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets tokenizers wandb evaluate optuna

In [2]:
import datasets
from datasets import load_dataset

from transformers import DefaultDataCollator
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

from transformers import pipeline
import json
from tqdm import tqdm, trange
from collections import defaultdict

In [3]:
SEED = 42
MODEL_NAME = "monologg/kobigbird-bert-base"

In [4]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    COLAB = True
except:
    COLAB = False

In [5]:
!unzip -o "/content/drive/MyDrive/Colab Notebooks/custom_dataset.zip"

'unzip'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [6]:
dataset_plus = load_dataset('ko_nia_normal_squad_all', split='train')
dataset_plus_fix = dataset_plus.filter(
    lambda example: example['question'].endswith(('는?', '가?', '은?', '나?', '요?'))
)

squad_plus = dataset_plus_fix.train_test_split(0.91, seed=SEED)
# filter: ('래?', '야?', '어?', '나?', '러?', '해?', '대?', '지')

squad_plus

Found cached dataset ko_nia_normal_squad_all (C:/Users/sms20/.cache/huggingface/datasets/ko_nia_normal_squad_all/ko_nia_normal_squad_all/0.0.1/f059be1cff43e2f3e3b26bf1def5c9f7672d505b4557ec67fcc447e69746ecdb)
Loading cached processed dataset at C:\Users\sms20\.cache\huggingface\datasets\ko_nia_normal_squad_all\ko_nia_normal_squad_all\0.0.1\f059be1cff43e2f3e3b26bf1def5c9f7672d505b4557ec67fcc447e69746ecdb\cache-bbef8f5762ec5d29.arrow
Loading cached split indices for dataset at C:\Users\sms20\.cache\huggingface\datasets\ko_nia_normal_squad_all\ko_nia_normal_squad_all\0.0.1\f059be1cff43e2f3e3b26bf1def5c9f7672d505b4557ec67fcc447e69746ecdb\cache-31de2438c36f9fac.arrow and C:\Users\sms20\.cache\huggingface\datasets\ko_nia_normal_squad_all\ko_nia_normal_squad_all\0.0.1\f059be1cff43e2f3e3b26bf1def5c9f7672d505b4557ec67fcc447e69746ecdb\cache-e90f09d99b3ad2ad.arrow


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 12211
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 123477
    })
})

In [7]:
dataset = load_dataset('custom_squad_v2', split='train')
squad = dataset.train_test_split(0.1, seed=SEED)

squad_augmented = datasets.concatenate_datasets([squad['train'], squad_plus['train']])
# squad['validation'] = squad['test']
# del squad['test']

Found cached dataset custom_squad_v2 (C:/Users/sms20/.cache/huggingface/datasets/custom_squad_v2/custom-squad_v2/0.0.1/429712f99e5bcf7342cc8873c4461efa3c29325d31e3770c03c761b9ade84e4e)
Loading cached split indices for dataset at C:\Users\sms20\.cache\huggingface\datasets\custom_squad_v2\custom-squad_v2\0.0.1\429712f99e5bcf7342cc8873c4461efa3c29325d31e3770c03c761b9ade84e4e\cache-93567b845d8c56f7.arrow and C:\Users\sms20\.cache\huggingface\datasets\custom_squad_v2\custom-squad_v2\0.0.1\429712f99e5bcf7342cc8873c4461efa3c29325d31e3770c03c761b9ade84e4e\cache-20d5ebf42d407cce.arrow


In [8]:
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10833
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1204
    })
})

In [9]:
squad_augmented

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 23044
})

In [10]:
print("Context: ", squad["train"][0]["context"])
print("Question: ", squad["train"][0]["question"])
print("Answer: ", squad["train"][0]["answers"])

Context:  SK텔레콤(대표이사 사장 박정호)은 20일부터 갤럭시S20 예약 판매에 돌입하고 오는 27일부터 개통을 시작한다고 밝혔다. 갤럭시S20 플러스 아우라 블루 컬러는 전 세계에서 오직 SK텔레콤에서만 구매 가능하다. 블루 컬러는 기존에 갤럭시 단말을 구매한 고객들로부터 큰 인기를 끌었다. SK텔레콤은 갤럭시S20 예약 판매를 맞아 온∙오프라인 채널에서 대대적인 ‘블루 마케팅’을 펼친다. 먼저 SK텔레콤 공식 온라인샵 T월드 다이렉트에서 갤럭시S20을 예약 및 구매한 고객 가운데 선착순 3천명에게 ‘아우라 블루 패키지’를 제공한다. ‘아우라 블루 패키지’는 갤럭시S20 단말과 아우라 블루 에코백, 신세계 상품권 3만원으로 구성돼 있다. T월드 다이렉트에서 예약한 모든 고객에게는 ▲PANTONE 액세서리 5종 ▲블루투스 이어폰 ▲맨프로토 미니 삼각대 ▲고속충전 지원 액세서리 총 8종의 사은품 가운데 1종을 증정한다. 또 예약 후 개통한 고객 가운데 100명을 추첨해 삼성 에어드레서, 무풍큐브, 블루스카이 공기청정기 등을 제공한다. 매장 방문이 어려운 고객은 T월드 다이렉트에서 ‘오늘도착’ 서비스를 신청하면, 원하는 시간과 장소에서 갤럭시S20을 손쉽게 받아 볼 수 있다. 전문 상담사가 직접 고객을 찾아가 개통부터 데이터 이전까지 마무리해 준다. 갤럭시S20을 저렴하게 구매하고 싶은 고객은 ▲제휴카드 더블할인 ▲T안심보상 ▲T모아쿠폰 등 다양한 구매 혜택을 활용해 최대 130만원까지 할인 받을 수 있다. 추후에도 새 갤럭시 단말로 기기변경을 원하는 고객은 ‘5GX 클럽 갤럭시S20’ 가입을 고려해 볼 만 하다. 월 이용료 7,480원(부가세 포함)을 내면 24개월 후 새 갤럭시 단말로 기기변경 시 출고가의 50% 수준까지 할인 받는다. SK텔레콤은 3월 멤버십 혜택으로 T Day에 ‘5GX 찬스’를 신설해 5GX 요금제 이용 고객을 대상으로 추가 혜택을 제공한다. 11번가 갤럭시S20 액세서리 22% 할인, 도미노피자 60% 할인 등 갤럭시S20 구매

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [12]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length= 3840, #2048, # electra: 386, bigbird: 1024
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [13]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Loading cached processed dataset at C:\Users\sms20\.cache\huggingface\datasets\custom_squad_v2\custom-squad_v2\0.0.1\429712f99e5bcf7342cc8873c4461efa3c29325d31e3770c03c761b9ade84e4e\cache-5aba2386c60c2c15.arrow


  0%|          | 0/2 [00:00<?, ?ba/s]

In [14]:
# tokenized_squad_augmented = squad_augmented.map(preprocess_function, batched=True, remove_columns=squad_augmented.column_names)

In [15]:
!pip install konlpy

from konlpy.tag import Okt

     --------------------------------------- 19.4/19.4 MB 50.4 MB/s eta 0:00:00
     ------------------------------------- 345.2/345.2 kB 20.9 MB/s eta 0:00:00


In [16]:
import re
def remove_josa(text):
    okt = Okt()
    while True:
        if not text:
            break
        cluttered = okt.pos(text)
        #print(cluttered)
        if cluttered[-1][1] == 'Josa' or cluttered[-1][1] == 'Eomi':
            text = text.rstrip(cluttered[-1][0])
        else:
            break
    if (text.startswith('‘') and text.endswith('’')) or (text.startswith('“') and text.endswith('”')):
        text = text.lstrip('‘').lstrip('“')
        text = text.rstrip('’').rstrip('”')
    regex = "\(.*\)|\s-\s.*"
    text = re.sub(regex, '', text)
    text = text.strip('◇').strip('○').strip('△')
    return text

In [17]:
data_collator = DefaultDataCollator()
models = []

#model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

# urls = '''https://huggingface.co/alphahg/kobigbird-base21-48297473
# https://huggingface.co/sudong97/kobigbird-base23-57995154
# https://huggingface.co/susooo/kobigbird-base27-63168558
# https://huggingface.co/sunny13/kobigbird-base32-27556755
# https://huggingface.co/sudong97/kobigbird-base24-6832218'''

urls ='''https://huggingface.co/sunny13/kobigbird-pure32-38554302
https://huggingface.co/alphahg/kobigbird-pure1-10335385
https://huggingface.co/alphahg/kobigbird-pure2-89302097
https://huggingface.co/YuJungSoo/kobigbird-pure25-64664366
https://huggingface.co/sunny13/kobigbird-pure31-53663328
https://huggingface.co/YuJungSoo/kobigbird-pure125-74195889
https://huggingface.co/sunny13/kobigbird-pure131-13172419
https://huggingface.co/susooo/kobigbird-pure147-92555918'''


urls = urls.split('\n')

for url in urls:
    name = url.split('huggingface.co/')[1]
    models.append(AutoModelForQuestionAnswering.from_pretrained(name))

models

Downloading:   0%|          | 0.00/930 [00:00<?, ?B/s]

c:\Users\sms20\miniconda3\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sms20\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/472M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/930 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/472M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/930 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/472M [00:00<?, ?B/s]

[BigBirdForQuestionAnswering(
   (bert): BigBirdModel(
     (embeddings): BigBirdEmbeddings(
       (word_embeddings): Embedding(32500, 768, padding_idx=0)
       (position_embeddings): Embedding(4096, 768)
       (token_type_embeddings): Embedding(2, 768)
       (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       (dropout): Dropout(p=0.1, inplace=False)
     )
     (encoder): BigBirdEncoder(
       (layer): ModuleList(
         (0): BigBirdLayer(
           (attention): BigBirdAttention(
             (self): BigBirdBlockSparseAttention(
               (query): Linear(in_features=768, out_features=768, bias=True)
               (key): Linear(in_features=768, out_features=768, bias=True)
               (value): Linear(in_features=768, out_features=768, bias=True)
             )
             (output): BigBirdSelfOutput(
               (dense): Linear(in_features=768, out_features=768, bias=True)
               (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [18]:
context = """\
BMW 코리아(대표 한상윤)는 창립 25주년을 기념하는 ‘BMW 코리아 25주년 에디션’을 한정 출시한다고 밝혔다. 이번 BMW 코리아 25주년 에디션(이하 25주년 에디션)은 BMW 3시리즈와 5시리즈, 7시리즈, 8시리즈 총 4종, 6개 모델로 출시되며, BMW 클래식 모델들로 선보인 바 있는 헤리티지 컬러가 차체에 적용돼 레트로한 느낌과 신구의 조화가 어우러진 차별화된 매력을 자랑한다. 먼저 뉴 320i 및 뉴 320d 25주년 에디션은 트림에 따라 옥스포드 그린(50대 한정) 또는 마카오 블루(50대 한정) 컬러가 적용된다. 럭셔리 라인에 적용되는 옥스포드 그린은 지난 1999년 3세대 3시리즈를 통해 처음 선보인 색상으로 짙은 녹색과 풍부한 펄이 오묘한 조화를 이루는 것이 특징이다. M 스포츠 패키지 트림에 적용되는 마카오 블루는 1988년 2세대 3시리즈를 통해 처음 선보인 바 있으며, 보랏빛 감도는 컬러감이 매력이다. 뉴 520d 25주년 에디션(25대 한정)은 프로즌 브릴리언트 화이트 컬러로 출시된다. BMW가 2011년에 처음 선보인 프로즌 브릴리언트 화이트는 한층 더 환하고 깊은 색감을 자랑하며, 특히 표면을 무광으로 마감해 특별함을 더했다. 뉴 530i 25주년 에디션(25대 한정)은 뉴 3시리즈 25주년 에디션에도 적용된 마카오 블루 컬러가 조합된다. 뉴 740Li 25주년 에디션(7대 한정)에는 말라카이트 그린 다크 색상이 적용된다. 잔잔하면서도 오묘한 깊은 녹색을 발산하는 말라카이트 그린 다크는 장식재로 활용되는 광물 말라카이트에서 유래됐다. 뉴 840i xDrive 그란쿠페 25주년 에디션(8대 한정)은 인도양의 맑고 투명한 에메랄드 빛을 연상케 하는 몰디브 블루 컬러로 출시된다. 특히 몰디브 블루는 지난 1993년 1세대 8시리즈에 처음으로 적용되었던 만큼 이를 오마주하는 의미를 담고 있다.
"""
question = "말라카이트에서 나온 색깔을 사용한 에디션은?"
question_answerer = pipeline("question-answering", model=models[0], tokenizer=tokenizer, device=0)
question_answerer(question=question, context=context)

Attention type 'block_sparse' is not possible if sequence_length: 384 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


{'score': 0.1705305427312851,
 'start': 666,
 'end': 682,
 'answer': '뉴 740Li 25주년 에디션'}

In [19]:
test_path = './custom_squad_v2/test.json'

test = []
with open(test_path, encoding="utf-8") as f:
    squad_ = json.load(f)
    for example in squad_["data"]:
        title = example.get("title", "")
        for paragraph in example["paragraphs"]:
            context = paragraph["context"]  # do not strip leading blank spaces GH-2585
            for qa in paragraph["qas"]:
                question = qa["question"]
                id_ = qa["guid"]

                # Features currently used are "context", "question", and "answers".
                # Others are extracted here for the ease of future expansions.
                test.append({
                    "title": title,
                    "context": context,
                    "question": question,
                    "id": id_,
                })

In [20]:
!pip install levenshtein

from Levenshtein import distance

def get_levenshtein(eval_preds_, answers):
    distances = []
    for i in range(len(eval_preds_)):
        for ans in answers[i]['text']:
            lev = distance(eval_preds_[i]['answer'], ans)
            distances.append(lev)

    mean_distance = sum(distances) / len(distances)
    
    return mean_distance

question_answerers = [pipeline("question-answering", model=model, tokenizer=tokenizer, device=0) for model in models]

In [21]:
def screen_pred(pred):
    length = len(pred['answer'])
    score = pred['score']
    cond = (length > 24 and score < 1.42) or (length > 16 and score < 0.36) or score < 0.02 or length > 60
    return cond

In [22]:
from copy import deepcopy

In [23]:
train_preds = []

for data in tqdm(squad['train']):
    predictions = [question_answerer(question=data['question'], context=data['context']) for question_answerer in question_answerers]
    
    vote = defaultdict(int)
    score = defaultdict(float)
    
    for prediction in predictions:
        prediction['answer'] = remove_josa(prediction['answer'])
        vote[prediction['answer']] += 1
        score[prediction['answer']] += prediction['score']

    maxi = -99999999999
    ans, same_vote = '', 0
    for key, value in vote.items():
        if maxi < value:
            maxi = value
            ans = key
            same_vote = 0
        elif maxi == value:
            same_vote += 1
    if same_vote != 0 or maxi <= 0:
        train_preds.append({'answer': '', 'score': 1})
    
    else:
        train_preds.append({'answer': remove_josa(ans), 'score': score[ans]/maxi})

  0%|          | 0/10833 [00:00<?, ?it/s]Attention type 'block_sparse' is not possible if sequence_length: 384 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...
Attention type 'block_sparse' is not possible if sequence_length: 384 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...
Attention type 'block_sparse' is not possible if sequence_length: 384 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_bl

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.

In [ ]:
train_preds_ = deepcopy(train_preds)
for i in range(len(train_preds)):
    pred = train_preds_[i]
    if screen_pred(pred):
        train_preds_[i]['answer'] = ''

In [ ]:
train_preds_

[{'answer': '7,480원', 'score': 0.9723370671272278},
 {'answer': '상점', 'score': 0.9279462695121765},
 {'answer': '528명', 'score': 0.8563057780265808},
 {'answer': '정종은', 'score': 0.9027838110923767},
 {'answer': '‘BMW 5시리즈’', 'score': 0.7828196287155151},
 {'answer': '70%', 'score': 0.9854837656021118},
 {'answer': '12월 7일부터', 'score': 0.88325035572052},
 {'answer': '저자극성 동물', 'score': 0.6664833426475525},
 {'answer': '각 지자체', 'score': 0.9266487956047058},
 {'answer': '5%', 'score': 0.6768468022346497},
 {'answer': '차으르', 'score': 0.4672701954841614},
 {'answer': '알칼리 전지', 'score': 0.8508975505828857},
 {'answer': '지리자동차', 'score': 0.6218090653419495},
 {'answer': '조건◇장사란 무엇', 'score': 0.4027042090892792},
 {'answer': '콘스탄티노폴리스 총대주교', 'score': 0.887279212474823},
 {'answer': '8천만', 'score': 0.15776990354061127},
 {'answer': '서울중앙지법', 'score': 0.9424945116043091},
 {'answer': '세라 아야코', 'score': 0.9581881165504456},
 {'answer': '', 'score': 0.006398413330316544},
 {'answer': '기획재정부', 'sco

In [ ]:
train_preds

[{'answer': '7,480원', 'score': 0.9723370671272278},
 {'answer': '상점', 'score': 0.9279462695121765},
 {'answer': '528명', 'score': 0.8563057780265808},
 {'answer': '정종은', 'score': 0.9027838110923767},
 {'answer': '‘BMW 5시리즈’', 'score': 0.7828196287155151},
 {'answer': '70%', 'score': 0.9854837656021118},
 {'answer': '12월 7일부터', 'score': 0.88325035572052},
 {'answer': '저자극성 동물', 'score': 0.6664833426475525},
 {'answer': '각 지자체', 'score': 0.9266487956047058},
 {'answer': '5%', 'score': 0.6768468022346497},
 {'answer': '차으르', 'score': 0.4672701954841614},
 {'answer': '알칼리 전지', 'score': 0.8508975505828857},
 {'answer': '지리자동차', 'score': 0.6218090653419495},
 {'answer': '조건◇장사란 무엇', 'score': 0.4027042090892792},
 {'answer': '콘스탄티노폴리스 총대주교', 'score': 0.887279212474823},
 {'answer': '8천만', 'score': 0.15776990354061127},
 {'answer': '서울중앙지법', 'score': 0.9424945116043091},
 {'answer': '세라 아야코', 'score': 0.9581881165504456},
 {'answer': '경북 영천', 'score': 0.006398413330316544},
 {'answer': '기획재정부',

In [ ]:
lev_score = get_levenshtein(train_preds_, squad['train']['answers'])
print(f'Train Lev Score: {lev_score}')

Train Lev Score: 3.235549405622995


In [ ]:
eval = []
eval_preds = []

for data in tqdm(squad['test']):
    predictions = [question_answerer(question=data['question'], context=data['context']) for question_answerer in question_answerers]
    
    vote = defaultdict(int)
    score = defaultdict(float)
    
    for prediction in predictions:
        prediction['answer'] = remove_josa(prediction['answer'])
        vote[prediction['answer']] += 1
        score[prediction['answer']] += prediction['score']

    maxi = -99999999999
    ans, same_vote = '', 0
    for key, value in vote.items():
        if maxi < value:
            maxi = value
            ans = key
            same_vote = 0
        elif maxi == value:
            same_vote += 1
    if same_vote != 0 or maxi <= 0:
        eval_preds.append({'answer': '', 'score': 1})
    
    else:
        eval_preds.append({'answer': remove_josa(ans), 'score': score[ans]/maxi})

100%|██████████| 1204/1204 [01:17<00:00, 15.52it/s]


In [ ]:
eval_preds_ = deepcopy(eval_preds)

for i in range(len(eval_preds)):
    pred = eval_preds_[i]
    if screen_pred(pred):
        eval_preds_[i]['answer'] = ''

In [ ]:
lev_score_test = get_levenshtein(eval_preds_, squad['test']['answers'])

print(f'Eval Lev Score: {lev_score_test}')

Eval Lev Score: 3.314625850340136


In [ ]:
eval_preds_

[{'answer': '자유', 'score': 0.052174292504787445},
 {'answer': '가솔린차가', 'score': 0.8214799761772156},
 {'answer': '르노삼성 XM3', 'score': 0.5184693336486816},
 {'answer': '234건', 'score': 0.19463711977005005},
 {'answer': '경원대군', 'score': 0.38969728350639343},
 {'answer': '140.7㎞', 'score': 0.10427191853523254},
 {'answer': '1997년이다', 'score': 0.9782876968383789},
 {'answer': '29억원', 'score': 0.9622801542282104},
 {'answer': '부천아이파크', 'score': 0.18432281911373138},
 {'answer': 'NC백화점', 'score': 0.6360383033752441},
 {'answer': 'AIA생명보험', 'score': 0.6837422847747803},
 {'answer': '10분', 'score': 0.776612401008606},
 {'answer': '마크다모르에서', 'score': 0.9219813346862793},
 {'answer': '믹수 파텔라이넨 감독', 'score': 0.5655525922775269},
 {'answer': '‘러비티’', 'score': 0.2943428158760071},
 {'answer': '컨트리(country)', 'score': 0.7535138726234436},
 {'answer': '모굴 칸', 'score': 0.4789780378341675},
 {'answer': '레몬즙, 파슬리', 'score': 0.5622847676277161},
 {'answer': '1건', 'score': 0.8828646540641785},
 {'answer':

In [ ]:
preds = []
# preds = [question_answerer(question=data['question'], context=data['context']) for data in tqdm(test)]

for data in tqdm(test):
    predictions = [question_answerer(question=data['question'], context=data['context']) for question_answerer in question_answerers]
    
    vote = defaultdict(int)
    score = defaultdict(float)
    
    for prediction in predictions:
        prediction['answer'] = remove_josa(prediction['answer'])
        vote[prediction['answer']] += 1
        score[prediction['answer']] += prediction['score']

    maxi = -99999999999
    ans, same_vote = '', 0
    for key, value in vote.items():
        if maxi < value:
            maxi = value
            ans = key
            same_vote = 0
        elif maxi == value:
            same_vote += 1
    if same_vote != 0 or maxi <= 0:
        preds.append({'answer': '', 'score': 1})
    
    else:
        preds.append({'answer': remove_josa(ans), 'score': score[ans]/maxi})

100%|██████████| 4008/4008 [04:17<00:00, 15.56it/s]


In [ ]:
preds

[{'answer': '뉴 740Li 25주년 에디션', 'score': 0.17043611407279968},
 {'answer': '독일 뒤셀도르프', 'score': 0.7233296632766724},
 {'answer': '링크트인과 페이스북', 'score': 0.7267038226127625},
 {'answer': '링크트인과 페이스북', 'score': 0.6679609417915344},
 {'answer': '마드리드', 'score': 0.9561583399772644},
 {'answer': '국제 원자재값 하락', 'score': 0.6178154945373535},
 {'answer': '5조원', 'score': 0.973518967628479},
 {'answer': '‘운영허가 인증서’', 'score': 0.7795740962028503},
 {'answer': '가열하여', 'score': 0.9302895665168762},
 {'answer': '노르웨이', 'score': 0.8739848732948303},
 {'answer': '22개', 'score': 0.8927226066589355},
 {'answer': '필리핀', 'score': 0.7182533740997314},
 {'answer': '김교성', 'score': 0.9909397959709167},
 {'answer': '60달러로', 'score': 0.1977325677871704},
 {'answer': '존 위클리프', 'score': 0.298484742641449},
 {'answer': '얀 후스', 'score': 0.49304214119911194},
 {'answer': '지기스문트', 'score': 0.34479743242263794},
 {'answer': '복정역', 'score': 0.39624351263046265},
 {'answer': '법무법인 한울', 'score': 0.376084566116333},
 {'answ

In [ ]:
test[0]['context'][666:682]

'뉴 740Li 25주년 에디션'

In [ ]:
import pandas as pd

df = pd.read_csv('blank.csv')
blanked = 0
for i in range(len(preds)):
    pred = preds[i]
    length = len(pred['answer'])
    if screen_pred(pred):
        df['Predicted'][i] = ''
        blanked += 1
    else:
        df['Predicted'][i] = pred['answer']

print(f'Blanked preds: {blanked}, total preds: {len(preds)}')

df = df.set_index('Id')

df.to_csv('pred.csv')

<ipython-input-55-968e2007ac26>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Predicted'][i] = pred['answer']
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Blanked preds: 56, total preds: 4008


In [ ]:
cnt = 0
for pred in preds:
    if pred['answer'] == '':
        cnt += 1

cnt

1

In [ ]:
# Optuna
import optuna

In [ ]:
def objective(trial):
    def screen_pred(pred):
        length = len(pred['answer'])
        cond = (length > trial.suggest_int('length1', 10, 40) and pred['score'] < trial.suggest_float('score1', 0.3, 1.5)) or ((length > trial.suggest_int('length2', 5, 30) and pred['score'] < trial.suggest_float('score2', 0.1, 0.8))) or pred['score'] < trial.suggest_float('minimum_score', 0, 0.2) or length > trial.suggest_int("max_length", 15, 100)
        return cond
        
    eval_preds_ = deepcopy(eval_preds)

    for i in range(len(eval_preds_)):
        pred = eval_preds_[i]
        if screen_pred(pred):
            eval_preds_[i]['answer'] = ''

    lev_score_test = get_levenshtein(eval_preds_, squad['test']['answers'])

    train_preds_ = deepcopy(train_preds)

    for i in range(len(train_preds_)):
        pred = train_preds_[i]
        length = len(pred['answer'])
        if screen_pred(pred):
            train_preds_[i]['answer'] = ''

    lev_score = get_levenshtein(train_preds_, squad['train']['answers'])

    return (lev_score_test*3 + lev_score) / 4

In [ ]:
study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_jobs=-1, n_trials=1000)

[I 2023-01-13 04:20:54,173] A new study created in memory with name: no-name-a6e197dc-e4f5-426e-8a62-0bbc2dfd338a
[I 2023-01-13 04:21:05,274] Trial 1 finished with value: 3.786602064563167 and parameters: {'length1': 10, 'length2': 12, 'minimum_score': 0.01163745097635549, 'max_length': 51, 'score1': 1.407603551881823}. Best is trial 1 with value: 3.786602064563167.
[I 2023-01-13 04:21:05,494] Trial 3 finished with value: 3.5286464985390342 and parameters: {'length1': 15, 'length2': 18, 'minimum_score': 0.10138560103162511, 'max_length': 41, 'score1': 1.3330775161183868}. Best is trial 3 with value: 3.5286464985390342.
[I 2023-01-13 04:21:05,849] Trial 0 finished with value: 3.322547962200164 and parameters: {'length1': 29, 'length2': 30, 'minimum_score': 0.021637433867400024, 'max_length': 90, 'score1': 1.3342489507020951}. Best is trial 0 with value: 3.322547962200164.
[I 2023-01-13 04:21:06,349] Trial 2 finished with value: 3.3183493175243557 and parameters: {'length1': 28, 'length2

In [ ]:
# You can use Matplotlib instead of Plotly for visualization by simply replacing `optuna.visualization` with
# `optuna.visualization.matplotlib` in the following examples.
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_intermediate_values(study)

[W 2023-01-13 05:11:42,383] You need to set up the pruning feature to utilize `plot_intermediate_values()`


In [ ]:
plot_parallel_coordinate(study)

In [ ]:
plot_contour(study)

In [ ]:
plot_slice(study)

In [ ]:
plot_param_importances(study)

In [ ]:
plot_edf(study)

In [ ]:
study.best_params

{'length1': 17,
 'length2': 27,
 'minimum_score': 0.04679462446913542,
 'max_length': 84,
 'score1': 0.3870818871055652,
 'score2': 0.44389623780985243}

In [ ]:
# [I 2023-01-13 04:47:05,531] Trial 509 finished with value: 3.2458631826842317 and parameters: {'length1': 17, 'length2': 27, 'minimum_score': 0.04679462446913542, 'max_length': 84, 'score1': 0.3870818871055652, 'score2': 0.44389623780985243}. Best is trial 509 with value: 3.2458631826842317.


In [ ]:
import pickle
import gzip


# save and compress.
with gzip.open('study1.pickle', 'wb') as f:
    pickle.dump(study, f)

In [ ]:
study.best_trial

FrozenTrial(number=509, values=[3.2458631826842317], datetime_start=datetime.datetime(2023, 1, 13, 4, 46, 53, 166321), datetime_complete=datetime.datetime(2023, 1, 13, 4, 47, 5, 531255), params={'length1': 17, 'length2': 27, 'minimum_score': 0.04679462446913542, 'max_length': 84, 'score1': 0.3870818871055652, 'score2': 0.44389623780985243}, distributions={'length1': IntDistribution(high=40, log=False, low=10, step=1), 'length2': IntDistribution(high=30, log=False, low=5, step=1), 'minimum_score': FloatDistribution(high=0.2, log=False, low=0.0, step=None), 'max_length': IntDistribution(high=100, log=False, low=15, step=1), 'score1': FloatDistribution(high=1.5, log=False, low=0.3, step=None), 'score2': FloatDistribution(high=0.8, log=False, low=0.1, step=None)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=509, state=TrialState.COMPLETE, value=None)